In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist
import matplotlib.pyplot as plt
import os
import ast
import tqdm
import faiss
from typing import Dict, List, Any, Tuple
from tqdm import tqdm
import random
import json

In [2]:
data_folder = 'C:/Data/Musicbrainz'

In [4]:
df = pd.read_csv(os.path.join(data_folder, 'musicbrainz-200-A01.csv'))
df

,TID,CID,CTID,SourceID,id,number,title,length,artist,album,year,language
0,1,1,1,1,WoM5452845,0,L'enfant aux yeux d'Italie (De vous à elle en ...,03:39,Daniel Balavoine,De vous à elle en passsant par moi,1975,NaN
1,2,32522,4,2,MBox36398328-HH,11,Silver Forest - 双剣乱舞 みょん＆me,229,NaN,Silver Forest 2006-2012 BEST1,12,Japanese
2,3,53749,2,3,4382873MB-01,2,shabnavard - Chavoush 2,27.183,شهرام ناظری,NaN,NaN,Persian
3,4,2,1,3,unk.,17,Mustard Gas - There and Back Again Lane,2.15,Action Painting!,NaN,'95,English
4,5,31368,3,4,215214-A048,6,006-Immer bis ich reier',3m 48sec,Nordwand,Das Pinke Album (unknown),NaN,Ger.
...,...,...,...,...,...,...,...,...,...,...,...,...
193745,193746,99998,1,4,159531-A065,2,002-Mayhem,5m 23sec,Kid Unknown,This Is Rough Mix (1992),NaN,Eng.
193746,193747,54645,2,1,WoM22726427,005,My Life (Blind Ravage),03:52,Blind Ravage,Blind Ravage,NaN,NaN
193747,193748,99997,2,5,14187740,24,"Sonata in D Minor for Cello and Piano, L 135: ...",216973,Claude Debussy,The 99 Most Essential Debussy Masterpieces,2010,NaN
193748,193749,99999,1,5,6431272,13,Make It Alone,184466,Powerplay,Ad Bouman's Goud Van Oud,1997,[Multiple languages]


In [5]:
df = df.astype({
    'TID': 'Int64',
    'CID': 'Int64',
    'CTID': 'Int32'})

In [6]:
nan_cols = ['SourceID', 'id', 'number', 'title', 'length', 'artist', 'album', 'year', 'language', ]
for col in nan_cols:
    df = df.astype({col: 'str'})
    df[col]  = df[col].apply(lambda x : '' if x == 'nan' else x)
df.head()

,TID,CID,CTID,SourceID,id,number,title,length,artist,album,year,language
0,1,1,1,1,WoM5452845,0,L'enfant aux yeux d'Italie (De vous à elle en ...,03:39,Daniel Balavoine,De vous à elle en passsant par moi,1975,
1,2,32522,4,2,MBox36398328-HH,11,Silver Forest - 双剣乱舞 みょん＆me,229,,Silver Forest 2006-2012 BEST1,12,Japanese
2,3,53749,2,3,4382873MB-01,2,shabnavard - Chavoush 2,27.183,شهرام ناظری,,,Persian
3,4,2,1,3,unk.,17,Mustard Gas - There and Back Again Lane,2.15,Action Painting!,,'95,English
4,5,31368,3,4,215214-A048,6,006-Immer bis ich reier',3m 48sec,Nordwand,Das Pinke Album (unknown),,Ger.


## Save the converted CSV
Save the CSV in a format that can be read by the **DuDe** program.

In [9]:
df.to_csv(os.path.join(data_folder, 'musicbrainz_200k_formatted.csv'), 
          index=False, 
          sep='\t')